In [1]:
# import packages
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import geopandas as gpd
import numpy as np
import glob
import rioxarray
import xarray

In [2]:
# create function to clip raster extent and save to file
def clip_raster(in_path_ras, in_path_vec, out_path, target_crs):
    
    # load vector data
    vector = gpd.read_file(in_path_vec)

    # with raster loaded
    with rasterio.open(in_path_ras) as src:
        
        # reproject vector to match raster CRS
        vector = vector.to_crs(src.crs)

        # convert vector geometry to GeoJSON format
        geometries = [geom for geom in vector.geometry]

        # clip raster
        clipped_raster, clipped_transform = mask(src, geometries, crop=True)

        # Calculate transform for the target CRS
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, clipped_raster.shape[2], clipped_raster.shape[1], *src.bounds
            )

        # update metadata
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "crs": target_crs,
            "height": height,
            "width": width,
            "transform": transform
        })

        # save clipped raster
        with rasterio.open(out_path, "w", **out_meta) as dest:
            reproject(
                source=clipped_raster,
                destination=rasterio.band(dest, 1),
                src_transform=clipped_transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=Resampling.nearest  
            )

    print(f"Clipped raster saved to {out_path} in CRS {target_crs}")

In [3]:
# create function to clip raster extent and save to file without reprojecting
def clip_raster_no_reprojection(in_path_ras, in_path_vec, out_path):

    # Load vector data
    vector = gpd.read_file(in_path_vec)

    # Open the raster file
    with rasterio.open(in_path_ras) as src:
        # Reproject the vector to match the raster CRS
        vector = vector.to_crs(src.crs)

        # Convert vector geometry to GeoJSON format
        geometries = [geom for geom in vector.geometry]

        # Clip the raster
        clipped_raster, clipped_transform = mask(src, geometries, crop=True)

        # Update the metadata
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": clipped_raster.shape[1],
            "width": clipped_raster.shape[2],
            "transform": clipped_transform
        })

        # Save the clipped raster
        with rasterio.open(out_path, "w", **out_meta) as dest:
            dest.write(clipped_raster)

    print(f"Clipped raster saved to {out_path} in original CRS {src.crs}")

In [4]:
# define function to read raster
def read_raster(name, as_type=float):
    """
    This is a simple function that reads the first band of the raster
    file "name" using rasterio. It is used in the function below.
    """
    r = rasterio.open(name)
    b = r.read(1)
    a = b.astype(as_type)
    return a

# define function to average rasters
def average_rasters(rasters, out_path):

    # Empty list to append input raster values
    all_read_rasters = []

    total_input_rasters = len(rasters)

    for r in range(0, total_input_rasters):
        rr = read_raster(rasters[r])
        all_read_rasters.append(rr)

    array_pixel_values = np.array(all_read_rasters)

    # 3D array, calculate along Z axis ("0")
    mean_array = np.nanmean(array_pixel_values, axis=0).round()

    # Open the first raster in order to get geoinfo!
    with rasterio.open(rasters[0]) as temp:
        # update metadata
        out_meta = temp.meta.copy()

    with rasterio.open(out_path, 'w', **out_meta) as mean_raster:
        mean_raster.write(mean_array, 1)
        mean_raster.close()

In [5]:
# set file paths 
# spawn 2010 datasets in paths
agb = './carbon_stock_data/cstock_spawn/agb_2010.tif'
agb_uncertainty = './carbon_stock_data/cstock_spawn/agb_uncertainty_2010.tif'
bgb = './carbon_stock_data/cstock_spawn/bgb_2010.tif'
bgb_uncertainty = './carbon_stock_data/cstock_spawn/bgb_uncertainty_2010.tif'

# spawn 2010 datasets out paths
agb_out = './carbon_stock_data/final_layers_clipped/agb_mg_ha_spawn_2010.tif'
agb_uncertainty_out = './carbon_stock_data/final_layers_clipped/agb_uncertainty_mg_ha_spawn_2010.tif'
bgb_out = './carbon_stock_data/final_layers_clipped/bgb_mg_ha_spawn_2010.tif'
bgb_uncertainty_out = './carbon_stock_data/final_layers_clipped/bgb_uncertainty_mg_ha_spawn_2010.tif'

# sothe datasets in paths
fc = './carbon_stock_data/cstock_canada_sothe/McMaster_WWFCanada_forest_carbon_250m/McMaster_WWFCanada_forest_carbon_250m_kg-m2_version1.0.tif'
fc_uncertainty = './carbon_stock_data/cstock_canada_sothe/McMaster_WWFCanada_forest_carbon_250m/McMaster_WWFCanada_forest_carbon_250m_kg-m2_uncertainty_version1.0.tif'
soc1m = './carbon_stock_data/cstock_canada_sothe/McMaster_WWFCanada_soil_carbon_250m_kg-m2_version3.0/McMaster_WWFCanada_soil_carbon1m_250m_kg-m2_version3.0.tif'
soc1m_uncertainty = './carbon_stock_data/cstock_canada_sothe/McMaster_WWFCanada_soil_carbon_250m_kg-m2_version3.0/McMaster_WWFCanada_soil_carbon1m_uncertainty_250m_kg-m2_version3.0.tif'

# sothe datasets out paths
fc_out = './carbon_stock_data/final_layers_clipped/forest_carbon_kg_m2_sothe.tif'
fc_uncertainty_out = './carbon_stock_data/final_layers_clipped/forest_carbon_uncertainty_kg_m2_sothe.tif'
soc1m_out = './carbon_stock_data/final_layers_clipped/soc_0_1m_kg_m2_sothe.tif'
soc1m_uncertainty_out = './carbon_stock_data/final_layers_clipped/soc_uncertainty_0_1m_kg_m2_sothe.tif'

# y2y boundary
y2y = './study_area/Y2Y_RegionBoundary_Final2013/Y2Y_RegionBoundary.shp'


In [6]:
# find center of y2y region
# Load the shapefile
gdf = gpd.read_file(y2y)

# Reproject to WGS84 if necessary
if gdf.crs != "EPSG:4326":
    gdf = gdf.to_crs("EPSG:4326")

# Get the bounding box (minx, miny, maxx, maxy)
minx, miny, maxx, maxy = gdf.total_bounds

# Calculate the center (midpoint of the bounding box)
center_lon = (minx + maxx) / 2
center_lat = (miny + maxy) / 2

print(f"Latitude: {center_lat}, Longitude: {center_lon}")

Latitude: 54.53624536240069, Longitude: -124.66760996293321


In [7]:
# define output CRS we want to use
# target_crs = '+proj=aea +lat_0=55 +lon_0=-125 +lat_1=42 +lat_2=68 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs'
target_crs = 'EPSG:4326'


In [51]:
# clip rasters
# # spawn
# clip_raster(agb, y2y, agb_out, target_crs)
# clip_raster(agb_uncertainty, y2y, agb_uncertainty_out, target_crs)
# clip_raster(bgb, y2y, bgb_out, target_crs)
# clip_raster(bgb_uncertainty, y2y, bgb_uncertainty_out, target_crs)

# spawn
clip_raster_no_reprojection(agb, y2y, agb_out)
clip_raster_no_reprojection(agb_uncertainty, y2y, agb_uncertainty_out)
clip_raster_no_reprojection(bgb, y2y, bgb_out)
clip_raster_no_reprojection(bgb_uncertainty, y2y, bgb_uncertainty_out)

# # sothe
# clip_raster(fc, y2y, fc_out, target_crs)
# clip_raster(fc_uncertainty, y2y, fc_uncertainty_out, target_crs)
# clip_raster(soc1m, y2y, soc1m_out, target_crs)
# clip_raster(soc1m_uncertainty, y2y, soc1m_uncertainty_out, target_crs)

# sothe
clip_raster_no_reprojection(fc, y2y, fc_out)
clip_raster_no_reprojection(fc_uncertainty, y2y, fc_uncertainty_out)
clip_raster_no_reprojection(soc1m, y2y, soc1m_out)
clip_raster_no_reprojection(soc1m_uncertainty, y2y, soc1m_uncertainty_out)


Clipped raster saved to ./carbon_stock_data/final_layers_clipped/agb_mg_ha_spawn_2010.tif in original CRS EPSG:4326
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/agb_uncertainty_mg_ha_spawn_2010.tif in original CRS EPSG:4326
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/bgb_mg_ha_spawn_2010.tif in original CRS EPSG:4326
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/bgb_uncertainty_mg_ha_spawn_2010.tif in original CRS EPSG:4326
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/forest_carbon_kg_m2_sothe.tif in original CRS EPSG:4326
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/forest_carbon_uncertainty_kg_m2_sothe.tif in original CRS EPSG:4326
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/soc_0_1m_kg_m2_sothe.tif in original CRS EPSG:4326
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/soc_uncertainty_0_1m_kg_m2_sothe.tif in original CRS EPSG:4326


In [8]:
# average rasters from years 2015-2017 of CONUS carbon
# because Sothe Canada layers are 2015-2019 (we only have until 2017 on the CONUS layers)
# function will output any data values available when one or more layers have no data in a pixel

# define folder path
conus_cstock_path = './carbon_stock_data/cstock_conus_cms/'

# create list of all raster names to clip
cms_conus_rasters = glob.glob(conus_cstock_path + '*.tif')
cms_conus_rasters


['./carbon_stock_data/cstock_conus_cms/CONUS_deadagb_2015_v1clipped.tif',
 './carbon_stock_data/cstock_conus_cms/CONUS_carbonlitter_2017_v1clipped.tif',
 './carbon_stock_data/cstock_conus_cms/CONUS_deadbgb_2017_v1clipped.tif',
 './carbon_stock_data/cstock_conus_cms/CONUS_deadagb_2016_v1clipped.tif',
 './carbon_stock_data/cstock_conus_cms/CONUS_carbonsoilorg_2017_v1clipped.tif',
 './carbon_stock_data/cstock_conus_cms/CONUS_bgb_2017_v1clipped.tif',
 './carbon_stock_data/cstock_conus_cms/CONUS_agb_2016_v1clipped.tif',
 './carbon_stock_data/cstock_conus_cms/CONUS_agb_2015_v1clipped.tif',
 './carbon_stock_data/cstock_conus_cms/CONUS_bgb_2015_v1clipped.tif',
 './carbon_stock_data/cstock_conus_cms/CONUS_bgb_2016_v1clipped.tif',
 './carbon_stock_data/cstock_conus_cms/CONUS_agb_2017_v1clipped.tif',
 './carbon_stock_data/cstock_conus_cms/CONUS_carbonlitter_2015_v1clipped.tif',
 './carbon_stock_data/cstock_conus_cms/CONUS_deadbgb_2016_v1clipped.tif',
 './carbon_stock_data/cstock_conus_cms/CONUS_c

In [46]:
# # clip rasters first for faster averaging of just our ROI
# for rast in cms_conus_rasters:
#     clip_raster(rast, y2y, rast.replace('.tif', 'clipped.tif'), target_crs)

for rast in cms_conus_rasters:
    clip_raster_no_reprojection(rast, y2y, rast.replace('.tif', 'clipped.tif'))

Clipped raster saved to ./carbon_stock_data/cstock_conus_cms/CONUS_deadagb_2015_v1clipped.tif in original CRS EPSG:5070
Clipped raster saved to ./carbon_stock_data/cstock_conus_cms/CONUS_deadbgb_2017_v1clipped.tif in original CRS EPSG:5070
Clipped raster saved to ./carbon_stock_data/cstock_conus_cms/CONUS_carbonlitter_2015_v1clipped.tif in original CRS EPSG:5070
Clipped raster saved to ./carbon_stock_data/cstock_conus_cms/CONUS_deadbgb_2015_v1clipped.tif in original CRS EPSG:5070
Clipped raster saved to ./carbon_stock_data/cstock_conus_cms/CONUS_deadagb_2017_v1clipped.tif in original CRS EPSG:5070
Clipped raster saved to ./carbon_stock_data/cstock_conus_cms/CONUS_carbonsoilorg_2016_v1clipped.tif in original CRS EPSG:5070
Clipped raster saved to ./carbon_stock_data/cstock_conus_cms/CONUS_carbonlitter_2017_v1clipped.tif in original CRS EPSG:5070
Clipped raster saved to ./carbon_stock_data/cstock_conus_cms/CONUS_agb_2017_v1clipped.tif in original CRS EPSG:5070
Clipped raster saved to ./ca

In [9]:
# list file names for rasters we want to average
agb_rasters = glob.glob(conus_cstock_path + '*_agb_????_v1clipped.tif')
print(agb_rasters)

bgb_rasters = glob.glob(conus_cstock_path + '*_bgb_????_v1clipped.tif')
print(bgb_rasters)

deadagb_rasters = glob.glob(conus_cstock_path + '*deadagb_????_v1clipped.tif')
print(deadagb_rasters)

deadbgb_rasters = glob.glob(conus_cstock_path + '*deadbgb_????_v1clipped.tif')
print(deadbgb_rasters)

litter_rasters = glob.glob(conus_cstock_path + '*litter_????_v1clipped.tif')
print(litter_rasters)

soc_rasters = glob.glob(conus_cstock_path + '*carbonsoilorg_????_v1clipped.tif')
print(soc_rasters)

['./carbon_stock_data/cstock_conus_cms/CONUS_agb_2016_v1clipped.tif', './carbon_stock_data/cstock_conus_cms/CONUS_agb_2015_v1clipped.tif', './carbon_stock_data/cstock_conus_cms/CONUS_agb_2017_v1clipped.tif']
['./carbon_stock_data/cstock_conus_cms/CONUS_bgb_2017_v1clipped.tif', './carbon_stock_data/cstock_conus_cms/CONUS_bgb_2015_v1clipped.tif', './carbon_stock_data/cstock_conus_cms/CONUS_bgb_2016_v1clipped.tif']
['./carbon_stock_data/cstock_conus_cms/CONUS_deadagb_2015_v1clipped.tif', './carbon_stock_data/cstock_conus_cms/CONUS_deadagb_2016_v1clipped.tif', './carbon_stock_data/cstock_conus_cms/CONUS_deadagb_2017_v1clipped.tif']
['./carbon_stock_data/cstock_conus_cms/CONUS_deadbgb_2017_v1clipped.tif', './carbon_stock_data/cstock_conus_cms/CONUS_deadbgb_2016_v1clipped.tif', './carbon_stock_data/cstock_conus_cms/CONUS_deadbgb_2015_v1clipped.tif']
['./carbon_stock_data/cstock_conus_cms/CONUS_carbonlitter_2017_v1clipped.tif', './carbon_stock_data/cstock_conus_cms/CONUS_carbonlitter_2015_v1c

In [48]:
# # average rasters for years 2015-2017
average_rasters(agb_rasters, './carbon_stock_data/final_layers_clipped/agb_mg_ha_cms_2015_2017.tif')
average_rasters(bgb_rasters, './carbon_stock_data/final_layers_clipped/bgb_mg_ha_cms_2015_2017.tif')
average_rasters(deadagb_rasters, './carbon_stock_data/final_layers_clipped/deadagb_mg_ha_cms_2015_2017.tif')
average_rasters(deadbgb_rasters, './carbon_stock_data/final_layers_clipped/deadbgb_mg_ha_cms_2015_2017.tif')
average_rasters(litter_rasters, './carbon_stock_data/final_layers_clipped/litter_mg_ha_cms_2015_2017.tif')
average_rasters(soc_rasters, './carbon_stock_data/final_layers_clipped/soc_mg_ha_cms_2015_2017.tif')

In [10]:
# list file names of soilgrids products
soc = './carbon_stock_data/soc_soilgrids/ocs_0-30cm_mean.tif'
soc_out = './carbon_stock_data/final_layers_clipped/soc_mean_0_30cm_t_ha_soilgrids.tif'

soc_05 = './carbon_stock_data/soc_soilgrids/ocs_0-30cm_Q0.05.tif'
soc_05_out = './carbon_stock_data/final_layers_clipped/soc_q05_0_30cm_t_ha_soilgrids.tif'

soc_95 = './carbon_stock_data/soc_soilgrids/ocs_0-30cm_Q0.95.tif'
soc_95_out = './carbon_stock_data/final_layers_clipped/soc_q95_0_30cm_t_ha_soilgrids.tif'

soc_uncertainty = './carbon_stock_data/soc_soilgrids/ocs_0-30cm_uncertainty.tif'
soc_uncertainty_out = './carbon_stock_data/final_layers_clipped/soc_uncertainty_0_30cm_t_ha_soilgrids.tif'

In [50]:
# # clip rasters
# clip_raster(soc, y2y, soc_out, target_crs)
# clip_raster(soc_05, y2y, soc_05_out, target_crs)
# clip_raster(soc_95, y2y, soc_95_out, target_crs)
# clip_raster(soc_uncertainty, y2y, soc_uncertainty_out, target_crs)

clip_raster_no_reprojection(soc, y2y, soc_out)
clip_raster_no_reprojection(soc_05, y2y, soc_05_out)
clip_raster_no_reprojection(soc_95, y2y, soc_95_out)
clip_raster_no_reprojection(soc_uncertainty, y2y, soc_uncertainty_out)

Clipped raster saved to ./carbon_stock_data/final_layers_clipped/soc_mean_0_30cm_t_ha_soilgrids.tif in original CRS PROJCS["Interrupted_Goode_Homolosine",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Interrupted_Goode_Homolosine"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/soc_q05_0_30cm_t_ha_soilgrids.tif in original CRS PROJCS["Interrupted_Goode_Homolosine",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Interrupted_Goode_Homolosine"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Clipped raster saved to ./carbon_stock_data/final_layers_clipped/soc_q9

In [ ]:
# moving on to the soc maps from openlandmap
# load file paths
soc_0_10 = './carbon_stock_data/soc_openlandmap/sol_organic.carbon.stock_msa.kgm2_m_250m_b0..10cm_1950..2017_v0.2.tif'
soc_10_30 = './carbon_stock_data/soc_openlandmap/sol_organic.carbon.stock_msa.kgm2_m_250m_b10..30cm_1950..2017_v0.2.tif'
soc_30_60 = './carbon_stock_data/soc_openlandmap/sol_organic.carbon.stock_msa.kgm2_m_250m_b30..60cm_1950..2017_v0.2.tif'
soc_60_100 = './carbon_stock_data/soc_openlandmap/sol_organic.carbon.stock_msa.kgm2_m_250m_b60..100cm_1950..2017_v0.2.tif'

In [ ]:
# first add all layers together
soc_dir = [soc_0_10, soc_10_30, soc_30_60, soc_60_100]

for i in range(0, len(soc_dir)):
    if i == 0:
        soc = xarray.open_dataarray(soc_dir[i])
    else:
        soc += xarray.open_dataarray(soc_dir[i])